In [94]:
import random

In [95]:
class Store():
    def __init__(self):
        self.animals = [
            Animal("Dog", 1, 2, 3),
            Animal("Cat", 1, 1, 2),
            Animal("Fish", 1, 1, 1),
            Animal("Bird", 1, 2, 2),
        ]

        self.display = [None] * 5

    def __str__(self):
        return f"Store (TIER: {self.tier}, ANIMALS: {', '.join(str(animal) for animal in self.animals)})"

    def _validate_int(self, value, name):
        if not isinstance(value, int):
            raise TypeError(f"{name} must be an integer")

    def _validate_non_negative(self, value, name):
        if value < 0:
            raise ValueError(f"{name} cannot be negative")

    def _validate_position(self, position):
        self._validate_int(position, "Position")
        if position < 0 or position >= len(self.team):
            raise ValueError("Position out of range")
        
    def add_animal(self, animal, position):
        self._validate_position(position)
        if self.display[position] is not None:
            raise ValueError("Position already occupied")
        if animal not in self.animals:
            raise ValueError("Animal not available in store")
        
        self.display[position] = animal

    def remove_animal(self, position):
        self._validate_position(position)
        if self.display[position] is None:
            raise ValueError("No animal to remove at this position")
        removed_animal = self.display[position]
        self.display[position] = None
        return removed_animal
    
    def choose_rand_animal(self):
        if not self.animals:
            raise ValueError("No animals available to choose from")
        return random.choice(self.animals)
    
    def populate_display(self):
        self.empty_display()
        for i in range(len(self.display)):
            self.display[i] = self.choose_rand_animal()

    def empty_display(self):
        for i in range(len(self.display)):
            self.display[i] = None

    def show_display(self):
        display_str = "Store Display:\n"
        display_str += "====================\n"
        # Gather string representations of animals in display
        animal_strs = []
        for animal in self.display:
            if animal is not None:
                animal_lines = str(animal).split('\n')
            else:
                animal_lines = [
                    "-------------------------",
                    " Name   : Empty",
                    " Tier   : -",
                    " Attack : -",
                    " Health : -",
                    " Cost   : -",
                    "-------------------------"
                ]
            animal_strs.append(animal_lines)
        # Transpose to stack horizontally
        for lines in zip(*animal_strs):
            display_str += "   ".join(lines) + "\n"
        return display_str
    
    def get_state(self):
        state = [None] * 5

        for i in range(5):
            pet = self.display[i]
            state[i] = {
                'is_empty': pet is None,
                'name': getattr(pet, 'name', None),
                'attack': getattr(pet, 'attack', None),
                'health': getattr(pet, 'health', None),
                'tier': getattr(pet, 'tier', None),
                'cost': getattr(pet, 'cost', None),
            }
            
        
        return state

class Animal:
    def __init__(self, name, tier, attack, health):
        self.name = name
        self.tier = tier
        self.attack = attack
        self.health = health
        self.cost = 3

    def __str__(self):
        out = (
            f"-------------------------\n"
            f" Name   : {self.name}\n"
            f" Tier   : {self.tier}\n"
            f" Attack : {self.attack}\n"
            f" Health : {self.health}\n"
            f" Cost   : {self.cost}\n"
            f"-------------------------"
        )
        return out

In [96]:
class Player():
    def __init__(self, name):
        self.name = name
        self.coins = 10
        self.wins = 0
        self.turn = 0
        self.hearts = 5
        self.is_alive = True

        self.team = [None] * 5

    def __str__(self):
        team_str = ', '.join(str(member) if member else "Empty" for member in self.team)
        return f"Player: {self.name}, Coins: {self.coins}, Wins: {self.wins}, Team: [{team_str}]"

    def _validate_int(self, value, name):
        if not isinstance(value, int):
            raise TypeError(f"{name} must be an integer")

    def _validate_non_negative(self, value, name):
        if value < 0:
            raise ValueError(f"{name} cannot be negative")

    def _validate_position(self, position):
        self._validate_int(position, "Position")
        if position < 0 or position >= len(self.team):
            raise ValueError("Position out of range")

    def add_coins(self, amount):
        self._validate_int(amount, "Amount")
        self._validate_non_negative(amount, "Amount")
        self.coins += amount

    def remove_coins(self, amount):
        self._validate_int(amount, "Amount")
        self._validate_non_negative(amount, "Amount")
        if self.coins < amount:
            raise ValueError("Not enough coins to remove")
        self.coins -= amount

    def add_member(self, animal, position):
        if all(member is not None for member in self.team):
            raise ValueError("Cannot add member to a full team")
        self._validate_position(position)
        if self.team[position] is not None:
            raise ValueError("Position already occupied")
        self.team[position] = animal

    def purchase_animal(self, animal, position):
        self._validate_position(position)
        if self.coins < animal.cost:
            raise ValueError("Not enough coins to purchase this animal")
        self.remove_coins(animal.cost)
        self.add_member(animal, position)

    def remove_member(self, position):
        self._validate_position(position)
        if self.team[position] is None:
            raise ValueError("No member to remove at this position")
        self.team[position] = None

    def get_state(self):
        state = [None] * 5

        for i in range(5):
            pet = self.team[i]
            state[i] = {
                'is_empty': pet is None,
                'name': getattr(pet, 'name', None),
                'attack': getattr(pet, 'attack', None),
                'health': getattr(pet, 'health', None),
                'tier': getattr(pet, 'tier', None),
                'cost': getattr(pet, 'cost', None),
            }
        
        return state

    def get_actions(self, state):
        actions = []
        coins = state["coins"]
        store = state["store"]
        team = state["team"]

        # --- Buy: for each store slot with an animal, each team slot with space ---
        for i, store_slot in enumerate(store):
            if not store_slot["is_empty"] and coins >= store_slot["cost"]:
                for j, team_slot in enumerate(team):
                    if team_slot["is_empty"]:
                        actions.append({"type": "buy", "store_slot": i, "team_slot": j})

        # --- Sell: for each non-empty team slot ---
        for i, team_slot in enumerate(team):
            if not team_slot["is_empty"]:
                actions.append({"type": "sell", "team_slot": i})

        # --- Reorder: swap between two non-empty, non-equal team slots ---
        for i in range(len(team)):
            for j in range(len(team)):
                if i != j and not team[i]["is_empty"] and not team[j]["is_empty"]:
                    actions.append({"type": "reorder", "source_slot": i, "target_slot": j})

        # --- Reroll: only if coins >= 1 ---
        if coins >= 1:
            actions.append({"type": "reroll"})

        # --- End turn: always legal ---
        actions.append({"type": "end"})

        return actions

    
    def decision_policy(self, state, actions):
        decision = {"type": "end"}
        return decision

    def execute(self, decision):
        ...

In [97]:
class Game():
    def __init__(self):
        self.players = {}
        self.store = Store()
        self.is_game_over = False
        self.n_max_turns = 10
        self.current_turn = 0

    def initialize_game(self):
        n_players = 4
        players = {
            "Player 1": Player("Player 1"),
            "Player 2": Player("Player 2"),
            "Player 3": Player("Player 3"),
            "Player 4": Player("Player 4"),
        }

        self.players = players
    
    def get_state(self, store, player):
        store_state = store.get_state()
        team_state = player.get_state()

        state = {
            'store' : store_state,
            'team' : team_state,
            'coins' : player.coins,
            'n_turn' : player.turn,
            'wins' : player.wins,
            'hearts' : player.hearts,
        }

        return state

    def prep_phase(self):
        print("Preparing game phase...")

        for player_id, player in self.players.items():
            print(f"> Preparing {player_id}...")

            self.store.populate_display()
            is_player_turn_over = False

            while not is_player_turn_over:
                state = self.get_state(self.store, player)
                actions = player.get_actions(state)
                decision = player.decision_policy(state, actions)

                # player.execute(decision)
                if decision['type'] == 'end':
                    is_player_turn_over = True
                
            print(f"> {player_id} is done with their turn.")

    def battle_phase(self):
        print("Battle phase...")

    def check_game_state(self):
        print("Checking game state...")
        if self.current_turn >= self.n_max_turns:
            self.is_game_over = True
        else:
            self.is_game_over = False
            

    def start_game(self):
        print("Starting game...")

        self.initialize_game()
        while not self.is_game_over:
            print(f"="*30)
            print(f"Turn {self.current_turn + 1}")
            print(f"#"*30)
            self.prep_phase()
            self.battle_phase()
            self.current_turn += 1
            self.check_game_state()

        print("Game over!")

            


In [98]:
game = Game()
game.start_game()

Starting game...
Turn 1
##############################
Preparing game phase...
> Preparing Player 1...
> Player 1 is done with their turn.
> Preparing Player 2...
> Player 2 is done with their turn.
> Preparing Player 3...
> Player 3 is done with their turn.
> Preparing Player 4...
> Player 4 is done with their turn.
Battle phase...
Checking game state...
Turn 2
##############################
Preparing game phase...
> Preparing Player 1...
> Player 1 is done with their turn.
> Preparing Player 2...
> Player 2 is done with their turn.
> Preparing Player 3...
> Player 3 is done with their turn.
> Preparing Player 4...
> Player 4 is done with their turn.
Battle phase...
Checking game state...
Turn 3
##############################
Preparing game phase...
> Preparing Player 1...
> Player 1 is done with their turn.
> Preparing Player 2...
> Player 2 is done with their turn.
> Preparing Player 3...
> Player 3 is done with their turn.
> Preparing Player 4...
> Player 4 is done with their turn.